In [2]:
import pandas as pd
from datetime import datetime
from dateutil.parser import parse
from peakdetect import peakdetect
import numpy as np
from plotly.subplots import make_subplots
import plotly.io as plotio
import plotly.graph_objects as gph
from scipy.signal import find_peaks
plotio.renderers.default='browser'
import plotly.express as px
from matplotlib import pyplot as plt
def findPattern(max_min):
    '''Function is used to find start and end dates and return the list of start and end dates of the patterns '''
    double_tops = []
    double_bottoms = []
    headshoulder=[]
    rev_headshoulder=[]
    rows=[double_tops,double_bottoms,headshoulder,rev_headshoulder]
   
    
    for i in range(5, len(max_min)):  
        window = max_min.iloc[i-5:i]
        
        ap1, bp2, cp3, dp4, ep5 = window.iloc[0:5,0]
        at,bt,ct,dt,et=window.iloc[0:5,1]
        
        if abs(bp2-ap1)>10 and abs(ep5-dp4)>10 and ap1<bp2 and cp3<ap1 and cp3<ep5 and cp3<dp4 and ep5<dp4 and abs(bp2-dp4)<=np.mean([bp2,dp4])*0.02 and at==bt and bt==ct and ct==dt and dt==et:
            rev_headshoulder.append((window.index[0], window.index[-1]))
           
        elif  abs(bp2-dp4) < np.mean([bp2,dp4])*0.05 and ap1>bp2 and ap1>dp4 and ep5>bp2 and ep5>dp4 and cp3>ap1 and cp3>bp2 and cp3>dp4 and cp3>bp2 and cp3>ep5 and at==bt and bt==ct and ct==dt and dt==et:
            headshoulder.append((window.index[0], window.index[-1]))
        
        elif abs(bp2-cp3)>=8 and abs(dp4-cp3)>=8 and abs(ap1-bp2)>=10 and abs(dp4-ep5)>=10 and ap1<bp2 and ap1<dp4 and cp3<bp2 and cp3<dp4 and ep5<bp2 and ep5<dp4 and cp3>ap1 and cp3>ep5 and abs(bp2-dp4)<=15 and at==bt and bt==ct and ct==dt and dt==et:
               double_tops.append((window.index[0], window.index[-1]))
#                print(window.index[0])
#                print(a)

        elif cp3<ep5 and cp3<ap1 and abs(bp2-cp3)>=8 and  ap1>bp2 and abs(dp4-cp3)>=8 and ap1>dp4 and cp3>bp2 and abs(ap1-bp2)>=10 and abs(dp4-ep5)>=10 and cp3>dp4 and ep5>bp2 and ep5>dp4 and abs(bp2-dp4)<=15 and at==bt and bt==ct and ct==dt and dt==et:
                double_bottoms.append((window.index[0], window.index[-1]))

    return double_tops, double_bottoms,headshoulder,rev_headshoulder, rows

def create_chart(df: pd.DataFrame):
    '''This Function is used to plot the Candlestick Chart'''
    layt = gph.Layout(
        title = 'D-Code',
        xaxis = { 'title': 'Time'},
        yaxis = { 'title': 'Price'},
    )
    
    img = make_subplots(rows=1, cols=1)
    
    img.add_trace(
        gph.Candlestick(
            x = df['date'],
            open = df['open'], 
            high = df['high'],
            low = df['low'],
            close = df['close'],
            name = 'Candlestick chart'
        ),
        row = 1,
        col = 1,
    )
    
    img.add_trace(
        gph.Scatter(
            x = upper_dates,
            y = upper_peaks,
            mode='markers',
            marker=dict(symbol='triangle-up', color='blue'),            
        ), row=1, col=1
    )
    
    
    img.add_trace(
        gph.Scatter(
            x = lower_dates,
            y = lower_peaks,
            mode='markers',
            marker=dict(symbol='triangle-down', color='red'),            
        ), row=1, col=1
    )
  
    
    img.update_xaxes(
        rangebreaks = [{'bounds': ['sat','mon']}],
        rangeslider_visible = False,
    )
    
    return img


df = pd.read_csv('TestData.csv')

'''Find The Local Maxima and Minima using peakdetect'''
peaks = peakdetect(df['close'],df['date'], lookahead=1) 

 
higherPeaks = (peaks[0])
lowerPeaks = (peaks[1])

upper_peaks = []
upper_dates = []

lower_peaks = []
lower_dates = []


for i in range(len(higherPeaks)):
    upper_dates.append(parse(higherPeaks[i][0]))
    upper_peaks.append(higherPeaks[i][1])
    
for i in range(len(lowerPeaks)):
    lower_dates.append(parse(lowerPeaks[i][0]))
    lower_peaks.append(lowerPeaks[i][1])
    
    

fig = create_chart(df)


'''Merging the points in one list converting it into a dataframe and sorting according to dates to get consecutive points'''
max_min = []
dates = []
date_formatted=[]
for j in range(len(peaks[0])):
    for i in range(len(peaks)):
        max_min.append(peaks[i][j][1])
        date_formatted.append(parse(peaks[i][j][0]).strftime("%m %d"))
        dates.append(peaks[i][j][0])
  
maxmin = pd.DataFrame(data={'col1':max_min,'col2':date_formatted},index=dates)
maxmin.sort_index() 

'''findPatterns is called to detect patterns and return an list of tuples containing start and end date'''
x,y,z,r,rows = findPattern(maxmin)
start_date=[]
end_date=[]
pattern=[]
startdt=[]
startdb=[]
starthsh=[]
startrhsh=[]

# print(maxmin.iloc[x[1][0]]['col2'])
# print(maxmin.iloc[x[1][1]]['col2'])
'''Creating a set of start dates of all the patterns'''
for i in range(len(x)):
    startdt.append(parse(x[i][0]).strftime('%y-%m-%d'))
    

for i in range(len(y)):
    startdb.append(parse(y[i][0]).strftime('%y-%m-%d'))
   
for i in range(len(z)):
    starthsh.append(parse(z[i][0]).strftime('%y-%m-%d'))
   
for i in range(len(r)):
    startrhsh.append(parse(r[i][0]).strftime('%y-%m-%d'))
'''Removing Redundant Patterns'''
startdt=set(startdt)
startdb=set(startdb)
starthsh=set(starthsh)
startrhsh=set(startrhsh)

startdt.difference_update(starthsh)
startdb.difference_update(startrhsh)
starthsh.difference_update(startrhsh)

print()
'''Checking if the date here is included in the start date set'''
for i in range(len(x)):
   if parse(x[i][0]).strftime('%y-%m-%d') in startdt:
        start_date.append(x[i][0])
        end_date.append(x[i][1])
        pattern.append('Double Top')

for i in range(len(y)):
   if parse(y[i][0]).strftime('%y-%m-%d') in startdb: 
        start_date.append(y[i][0])
        end_date.append(y[i][1])
        pattern.append('Double Bottom')

for i in range(len(z)): 
    if parse(z[i][0]).strftime('%y-%m-%d') in starthsh:
        start_date.append(z[i][0])
        end_date.append(z[i][1])
        pattern.append('Head and Shoulder')
for i in range(len(r)):

    start_date.append(r[i][0])
    end_date.append(r[i][1])
    pattern.append('Reverse Head and Shoulder')
    
    
final_dict={'start':start_date,'end':end_date,'pattern':pattern}
final=pd.DataFrame(final_dict)
final.sort_values(["start"],axis=0, ascending=True,inplace=True,na_position='first')
final.to_csv('TestDataDT.csv') 
fig.show()